# model

## cnn fnn

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from importlib import import_module
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

class candle_model(pl.LightningModule):
    def __init__(self, input_dim, output_channel, num_classes, lr=1e-3, threshold=0.5, label_weights_tensor=None,
                 scheduler_name=None, scheduler_params=None, optimizer_params=None, optimizer_name="adamw", dropout =0.2):
        super().__init__()
        self.save_hyperparameters()
        self.optimizer_name = optimizer_name
        self.scheduler_name = scheduler_name or None
        self.optimizer_params = optimizer_params or {}
        self.scheduler_params = scheduler_params or {}
        self.threshold = threshold
        self.lr = lr
        # Enhanced Feature Extractor
        self.feature_extractor = nn.Sequential(
            # 1D convolution: in_channels=input_dim, out_channels=output_channels
            nn.Conv1d(in_channels=input_dim, out_channels=output_channel, kernel_size=1),
            nn.BatchNorm1d(output_channel),  # Added Batch Normalization
            nn.ReLU(),
            nn.Dropout(dropout)  # Added Dropout
        )

        # Linear classifier
        self.fc = nn.Linear(output_channel, num_classes)

        # Loss with optional weights
        if label_weights_tensor is not None:
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=label_weights_tensor)
        else:
            self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        # x: (batch, seq_len=1, input_dim)
        x = x.transpose(1, 2)  # (batch, input_dim, 1)
        out = self.feature_extractor(x)  # Pass through the sequential block
        out = out.squeeze(2)  # (batch, output_channels)
        logits = self.fc(out)  # (batch, num_classes)
        return logits
    
    def validation_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        probs = torch.sigmoid(logits)
        preds = (probs >= self.threshold).float()
        exact_acc = (preds == y_multi).all(dim=1).float().mean()

        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc_exact", exact_acc, prog_bar=True, on_epoch=True)

        return {"val_loss": loss, "probs": probs, "targets": y_multi}
    
    def training_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        return loss
    
    def configure_optimizers(self):
        """
        Configures the optimizer and learning rate scheduler based on model parameters.
        """
        # 1. Map string names to PyTorch classes
        optimizer_map = {
            "adamw": optim.AdamW,
            "adam": optim.Adam,
            # Add other optimizers here as needed
        }
        scheduler_map = {
            "reduce_on_plateau": lr_scheduler.ReduceLROnPlateau,
            "step_lr": lr_scheduler.StepLR,
            "onecycle": lr_scheduler.OneCycleLR,
            # Add other schedulers here as needed
        }

        # 2. Get optimizer instance
        optimizer_class = optimizer_map[self.optimizer_name]
        optimizer = optimizer_class(self.parameters(), lr=self.lr, **self.optimizer_params)

        # 3. Handle the "no scheduler" case
        if self.scheduler_name is None or self.scheduler_name.lower() == "none":
            return optimizer

        # 4. Get scheduler instance with proper parameters
        scheduler_class = scheduler_map[self.scheduler_name]

        # Handle one-off cases like OneCycleLR which requires a specific total_steps
        if self.scheduler_name == "onecycle":
            scheduler = scheduler_class(
                optimizer,
                max_lr=self.lr,
                total_steps=self.trainer.estimated_stepping_batches, # Requires PyTorch Lightning 2.0+
                **self.scheduler_params
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "interval": "step", # OneCycleLR updates on each step
                },
            }
        
        # Handle ReduceLROnPlateau, which requires monitoring a metric
        if self.scheduler_name == "reduce_on_plateau":
            scheduler = scheduler_class(optimizer, **self.scheduler_params)
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "monitor": "val_loss", # Monitor a specific validation metric
                },
            }

        # Handle all other schedulers (e.g., StepLR)
        scheduler = scheduler_class(optimizer, **self.scheduler_params)
        return [optimizer], [scheduler]


## fnn cnn fnn

In [69]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from importlib import import_module
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
class candle_model(pl.LightningModule):
    def __init__(self, input_dim, fnn_hidden_dim, output_channel, num_classes, lr=1e-3, threshold=0.5, label_weights_tensor=None,
                 scheduler_name=None, scheduler_params=None, optimizer_params=None, optimizer_name="adamw", dropout=0.2):
        super().__init__()
        self.save_hyperparameters()
        self.optimizer_name = optimizer_name
        self.scheduler_name = scheduler_name or None
        self.optimizer_params = optimizer_params or {}
        self.scheduler_params = scheduler_params or {}
        self.threshold = threshold
        self.lr = lr

        self.fnn_layer = nn.Sequential(
            nn.Linear(input_dim, fnn_hidden_dim),
            nn.BatchNorm1d(fnn_hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        self.cnn_extractor = nn.Sequential(
            nn.Conv1d(in_channels=fnn_hidden_dim, out_channels=output_channel, kernel_size=1),
            nn.BatchNorm1d(output_channel),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        self.fc = nn.Linear(output_channel, num_classes)

        if label_weights_tensor is not None:
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=label_weights_tensor)
        else:
            self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = x.squeeze(1) # (batch, input_dim)
        x = self.fnn_layer(x) # (batch, fnn_hidden_dim)
        x = x.unsqueeze(2) # (batch, fnn_hidden_dim, 1) - reshape for Conv1d
        out = self.cnn_extractor(x)
        out = out.squeeze(2) # (batch, output_channel)
        logits = self.fc(out)
        return logits

    def training_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        probs = torch.sigmoid(logits)
        preds = (probs >= self.threshold).float()
        exact_acc = (preds == y_multi).all(dim=1).float().mean()
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc_exact", exact_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": loss, "probs": probs, "targets": y_multi}

    def configure_optimizers(self):
        """
        Configures the optimizer and learning rate scheduler based on model parameters.
        """
        # 1. Map string names to PyTorch classes
        optimizer_map = {
            "adamw": optim.AdamW,
            "adam": optim.Adam,
            # Add other optimizers here as needed
        }
        scheduler_map = {
            "reduce_on_plateau": lr_scheduler.ReduceLROnPlateau,
            "step_lr": lr_scheduler.StepLR,
            "onecycle": lr_scheduler.OneCycleLR,
            # Add other schedulers here as needed
        }

        # 2. Get optimizer instance
        optimizer_class = optimizer_map[self.optimizer_name]
        optimizer = optimizer_class(self.parameters(), lr=self.lr, **self.optimizer_params)

        # 3. Handle the "no scheduler" case
        if self.scheduler_name is None or self.scheduler_name.lower() == "none":
            return optimizer

        # 4. Get scheduler instance with proper parameters
        scheduler_class = scheduler_map[self.scheduler_name]

        # Handle one-off cases like OneCycleLR which requires a specific total_steps
        if self.scheduler_name == "onecycle":
            scheduler = scheduler_class(
                optimizer,
                max_lr=self.lr,
                total_steps=self.trainer.estimated_stepping_batches, # Requires PyTorch Lightning 2.0+
                **self.scheduler_params
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "interval": "step", # OneCycleLR updates on each step
                },
            }
        
        # Handle ReduceLROnPlateau, which requires monitoring a metric
        if self.scheduler_name == "reduce_on_plateau":
            scheduler = scheduler_class(optimizer, **self.scheduler_params)
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "monitor": "val_loss", # Monitor a specific validation metric
                },
            }

        # Handle all other schedulers (e.g., StepLR)
        scheduler = scheduler_class(optimizer, **self.scheduler_params)
        return [optimizer], [scheduler]

## full fnn

In [2]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from importlib import import_module
from sklearn.metrics import f1_score, accuracy_score
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

class candle_model(pl.LightningModule):
    def __init__(self, input_dim,num_classes, hidden_dim =10, lr=1e-3, threshold=0.5, label_weights_tensor=None,
                 scheduler_name=None, scheduler_params=None, optimizer_params=None, optimizer_name="adamw", dropout=0.2):
        super().__init__()
        self.save_hyperparameters()
        self.optimizer_name = optimizer_name
        self.scheduler_name = scheduler_name or None
        self.optimizer_params = optimizer_params or {}
        self.scheduler_params = scheduler_params or {}
        self.threshold = threshold
        self.lr = lr

        self.fcn = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )

        if label_weights_tensor is not None:
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=label_weights_tensor)
        else:
            self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        logits = self.fcn(x.squeeze(1)) # Squeeze the seq_len=1 dimension
        return logits

    def training_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        probs = torch.sigmoid(logits)
        preds = (probs >= self.threshold).float()
        
        # Calculate exact accuracy
        exact_acc = (preds == y_multi).all(dim=1).float().mean()
        
        # Convert tensors to numpy arrays for sklearn metrics
        y_true_np = y_multi.cpu().numpy()
        y_preds_np = preds.cpu().numpy()
        
        # Calculate F1-macro and micro accuracy
        f1_macro = f1_score(y_true_np, y_preds_np, average="macro", zero_division=0)
        acc_micro = accuracy_score(y_true_np, y_preds_np)
        
        # Log the metrics to Lightning
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log("val_acc_exact", exact_acc, prog_bar=True, on_epoch=True)
        self.log("val_acc_micro", acc_micro, prog_bar=True, on_epoch=True)
        self.log("val_f1_macro", f1_macro, prog_bar=True, on_epoch=True)

        return {"val_loss": loss, "probs": probs, "targets": y_multi}
        
    def configure_optimizers(self):
        """
        Configures the optimizer and learning rate scheduler based on model parameters.
        """
        # 1. Map string names to PyTorch classes
        optimizer_map = {
            "adamw": optim.AdamW,
            "adam": optim.Adam,
            # Add other optimizers here as needed
        }
        scheduler_map = {
            "reduce_on_plateau": lr_scheduler.ReduceLROnPlateau,
            "step_lr": lr_scheduler.StepLR,
            "onecycle": lr_scheduler.OneCycleLR,
            # Add other schedulers here as needed
        }

        # 2. Get optimizer instance
        optimizer_class = optimizer_map[self.optimizer_name]
        optimizer = optimizer_class(self.parameters(), lr=self.lr, **self.optimizer_params)

        # 3. Handle the "no scheduler" case
        if self.scheduler_name is None or self.scheduler_name.lower() == "none":
            return optimizer

        # 4. Get scheduler instance with proper parameters
        scheduler_class = scheduler_map[self.scheduler_name]

        # Handle one-off cases like OneCycleLR which requires a specific total_steps
        if self.scheduler_name == "onecycle":
            scheduler = scheduler_class(
                optimizer,
                max_lr=self.lr,
                total_steps=self.trainer.estimated_stepping_batches, # Requires PyTorch Lightning 2.0+
                **self.scheduler_params
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "interval": "step", # OneCycleLR updates on each step
                },
            }
        
        # Handle ReduceLROnPlateau, which requires monitoring a metric
        if self.scheduler_name == "reduce_on_plateau":
            scheduler = scheduler_class(optimizer, **self.scheduler_params)
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "monitor": "val_loss", # Monitor a specific validation metric
                },
            }

        # Handle all other schedulers (e.g., StepLR)
        scheduler = scheduler_class(optimizer, **self.scheduler_params)
        return [optimizer], [scheduler]

## cnn lstm (seq)

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

class candle_cnn_multi_lstm(pl.LightningModule):
    def __init__(
        self,
        input_dim,
        num_classes,
        kernels=[1, 3, 5],
        cnn_out_channels=32,
        fusion_out_channels=64,
        hidden_dim=128,
        num_lstm_layers=1,
        lr=1e-3,
        threshold=0.5,
        label_weights_tensor=None,
        scheduler_name=None,
        scheduler_params=None,
        optimizer_params=None,
        optimizer_name="adamw",
        dropout=0.2,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.optimizer_name = optimizer_name
        self.scheduler_name = scheduler_name or None
        self.optimizer_params = optimizer_params or {}
        self.scheduler_params = scheduler_params or {}
        self.threshold = threshold
        self.lr = lr

        self.input_dim = input_dim
        self.kernels = kernels
        self.num_branches = len(kernels)
        self.cnn_out_channels = cnn_out_channels
        self.fusion_out_channels = fusion_out_channels

        # --- Multi-branch convolutions ---
        branches = []
        for k in kernels:
            pad = (k - 1) // 2  # approximate "same" padding
            branches.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=input_dim,
                        out_channels=cnn_out_channels,
                        kernel_size=k,
                        padding=pad,
                    ),
                    nn.BatchNorm1d(cnn_out_channels),
                    nn.ReLU(inplace=True),
                    nn.Dropout(dropout),
                )
            )
        self.branches = nn.ModuleList(branches)

        # --- Fusion conv2d ---
        # After stacking, shape = (B, num_branches, C, T)
        # We want kernel height = cnn_out_channels to "collapse" feature dim
        self.fusion_conv2d = nn.Sequential(
            nn.Conv2d(
                in_channels=self.num_branches,
                out_channels=self.fusion_out_channels,
                kernel_size=(self.cnn_out_channels, 1),
                padding=(0, 0),
            ),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
        )

        # --- LSTM ---
        self.lstm = nn.LSTM(
            input_size=self.fusion_out_channels,
            hidden_size=hidden_dim,
            num_layers=num_lstm_layers,
            batch_first=True,
            dropout=dropout if num_lstm_layers > 1 else 0,
            bidirectional=False,
        )

        # --- Final classifier ---
        self.fc = nn.Linear(hidden_dim, num_classes)

        # --- Loss ---
        if label_weights_tensor is not None:
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=label_weights_tensor)
        else:
            self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        # x: (B, seq_len, input_dim)
        x = x.transpose(1, 2)  # -> (B, input_dim, T)

        # Each branch processes the same input
        branch_outs = [branch(x) for branch in self.branches]  # list of (B, C, T)

        # Stack into (B, num_branches, C, T)
        stacked = torch.stack(branch_outs, dim=1)

        # Fusion conv2d expects (B, num_branches, C, T)
        fused = self.fusion_conv2d(stacked)  # -> (B, fusion_out_channels, 1, T)

        fused = fused.squeeze(2).transpose(1, 2)  # -> (B, T, fusion_out_channels)

        # LSTM
        lstm_out, _ = self.lstm(fused)  # -> (B, T, hidden_dim)
        last_hidden = lstm_out[:, -1, :]  # (B, hidden_dim)

        logits = self.fc(last_hidden)  # (B, num_classes)
        return logits

    def training_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_multi = batch
        logits = self(x)
        loss = self.criterion(logits, y_multi.float())
        probs = torch.sigmoid(logits)
        preds = (probs >= self.threshold).float()
        exact_acc = (preds == y_multi).all(dim=1).float().mean()

        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc_exact", exact_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": loss, "probs": probs, "targets": y_multi}

    def configure_optimizers(self):
        optimizer_map = {
            "adamw": optim.AdamW,
            "adam": optim.Adam,
        }
        scheduler_map = {
            "reduce_on_plateau": lr_scheduler.ReduceLROnPlateau,
            "step_lr": lr_scheduler.StepLR,
            "onecycle": lr_scheduler.OneCycleLR,
        }

        optimizer_class = optimizer_map[self.optimizer_name]
        optimizer = optimizer_class(self.parameters(), lr=self.lr, **self.optimizer_params)

        if self.scheduler_name is None or self.scheduler_name.lower() == "none":
            return optimizer

        scheduler_class = scheduler_map[self.scheduler_name]

        if self.scheduler_name == "onecycle":
            scheduler = scheduler_class(
                optimizer,
                max_lr=self.lr,
                total_steps=self.trainer.estimated_stepping_batches,
                **self.scheduler_params,
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {"scheduler": scheduler, "interval": "step"},
            }

        if self.scheduler_name == "reduce_on_plateau":
            scheduler = scheduler_class(optimizer, **self.scheduler_params)
            return {
                "optimizer": optimizer,
                "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"},
            }

        scheduler = scheduler_class(optimizer, **self.scheduler_params)
        return [optimizer], [scheduler]


## full fnn (seq)

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

class candle_fnn(pl.LightningModule):
    def __init__(self, input_dim, seq_len, hidden_dim=128, num_classes=2,
                 lr=1e-3, threshold=0.5, label_weights_tensor=None,
                 scheduler_name=None, scheduler_params=None,
                 optimizer_params=None, optimizer_name="adamw", dropout=0.2):
        super().__init__()
        self.save_hyperparameters()

        self.seq_len = seq_len
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.lr = lr
        self.threshold = threshold
        self.optimizer_name = optimizer_name
        self.scheduler_name = scheduler_name
        self.optimizer_params = optimizer_params or {}
        self.scheduler_params = scheduler_params or {}

        # Flatten input to (B, seq_len * input_dim)
        self.fc_layers = nn.Sequential(
            nn.Linear(seq_len * input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )

        if label_weights_tensor is not None:
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=label_weights_tensor)
        else:
            self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        # x: (B, seq_len, input_dim)
        x = x.view(x.size(0), -1)  # Flatten
        logits = self.fc_layers(x)
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y.float())
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y.float())
        probs = torch.sigmoid(logits)
        preds = (probs >= self.threshold).float()
        exact_acc = (preds == y).all(dim=1).float().mean()
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc_exact", exact_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": loss, "probs": probs, "targets": y}

    def configure_optimizers(self):
        optimizer_map = {"adamw": optim.AdamW, "adam": optim.Adam}
        scheduler_map = {
            "reduce_on_plateau": lr_scheduler.ReduceLROnPlateau,
            "step_lr": lr_scheduler.StepLR,
            "onecycle": lr_scheduler.OneCycleLR,
        }

        optimizer = optimizer_map[self.optimizer_name](self.parameters(), lr=self.lr, **self.optimizer_params)

        if self.scheduler_name is None or self.scheduler_name.lower() == "none":
            return optimizer

        scheduler_class = scheduler_map[self.scheduler_name]

        if self.scheduler_name == "onecycle":
            scheduler = scheduler_class(
                optimizer,
                max_lr=self.lr,
                total_steps=self.trainer.estimated_stepping_batches,
                **self.scheduler_params
            )
            return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "interval": "step"}}

        if self.scheduler_name == "reduce_on_plateau":
            scheduler = scheduler_class(optimizer, **self.scheduler_params)
            return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"}}

        scheduler = scheduler_class(optimizer, **self.scheduler_params)
        return [optimizer], [scheduler]


## cnn fnn (seq)

In [ ]:
class candle_cnn_fnn(pl.LightningModule):
    def __init__(self, input_dim, seq_len, cnn_out_channels=32, hidden_dim=128, num_classes=2,
                 kernels=[1,3,5], lr=1e-3, threshold=0.5, label_weights_tensor=None,
                 scheduler_name=None, scheduler_params=None,
                 optimizer_params=None, optimizer_name="adamw", dropout=0.2):
        super().__init__()
        self.save_hyperparameters()
        self.input_dim = input_dim
        self.seq_len = seq_len
        self.cnn_out_channels = cnn_out_channels
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.kernels = [k for k in kernels if k <= seq_len]  # sanity check
        self.lr = lr
        self.threshold = threshold
        self.optimizer_name = optimizer_name
        self.scheduler_name = scheduler_name
        self.optimizer_params = optimizer_params or {}
        self.scheduler_params = scheduler_params or {}

        # --- Multi-kernel CNN branches ---
        self.branches = nn.ModuleList()
        for k in self.kernels:
            pad = (k - 1) // 2
            self.branches.append(
                nn.Sequential(
                    nn.Conv1d(in_channels=input_dim, out_channels=cnn_out_channels, kernel_size=k, padding=pad),
                    nn.BatchNorm1d(cnn_out_channels),
                    nn.ReLU(),
                    nn.Dropout(dropout)
                )
            )

        # LSTM after CNN
        self.lstm = nn.LSTM(input_size=cnn_out_channels * len(self.kernels),
                            hidden_size=hidden_dim,
                            batch_first=True)

        # Final linear layer
        self.fc = nn.Linear(hidden_dim, num_classes)

        if label_weights_tensor is not None:
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=label_weights_tensor)
        else:
            self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        # x: (B, seq_len, input_dim)
        x = x.transpose(1,2)  # (B, input_dim, seq_len)
        branch_outs = [branch(x) for branch in self.branches]  # list of (B, C, seq_len)
        x_cat = torch.cat(branch_outs, dim=1)  # (B, C*num_branches, seq_len)
        x_cat = x_cat.transpose(1,2)  # (B, seq_len, C*num_branches)
        lstm_out, _ = self.lstm(x_cat)  # (B, seq_len, hidden_dim)
        logits = self.fc(lstm_out[:, -1, :])  # use last time step
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y.float())
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y.float())
        probs = torch.sigmoid(logits)
        preds = (probs >= self.threshold).float()
        exact_acc = (preds == y).all(dim=1).float().mean()
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc_exact", exact_acc, prog_bar=True, on_epoch=True)
        return {"val_loss": loss, "probs": probs, "targets": y}

    def configure_optimizers(self):
        optimizer_map = {"adamw": optim.AdamW, "adam": optim.Adam}
        scheduler_map = {
            "reduce_on_plateau": lr_scheduler.ReduceLROnPlateau,
            "step_lr": lr_scheduler.StepLR,
            "onecycle": lr_scheduler.OneCycleLR,
        }

        optimizer = optimizer_map[self.optimizer_name](self.parameters(), lr=self.lr, **self.optimizer_params)

        if self.scheduler_name is None or self.scheduler_name.lower() == "none":
            return optimizer

        scheduler_class = scheduler_map[self.scheduler_name]

        if self.scheduler_name == "onecycle":
            scheduler = scheduler_class(
                optimizer,
                max_lr=self.lr,
                total_steps=self.trainer.estimated_stepping_batches,
                **self.scheduler_params
            )
            return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "interval": "step"}}

        if self.scheduler_name == "reduce_on_plateau":
            scheduler = scheduler_class(optimizer, **self.scheduler_params)
            return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"}}

        scheduler = scheduler_class(optimizer, **self.scheduler_params)
        return [optimizer], [scheduler]


# neural network

## lstm

In [ ]:
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from datetime import datetime
import pandas as pd
import io
import os
import numpy as np
from preprocess.multilabel_preprocess2 import preprocess_csv_multilabel
from models.LSTM.lstm_multi_label import LSTMMultiLabelClassifier
from utils.print_batch import print_batch
from utils.json_to_csv import json_to_csv_in_memory  # <-- new util
from utils.multilabel_threshold_tuning import tune_thresholds_nn
from add_ons.feature_pipeline5 import FeaturePipeline
from add_ons.drop_columns2 import drop_columns
from add_ons.candle_dif_rate_of_change_percentage2 import add_candle_rocp
from add_ons.candle_proportion import add_candle_proportions
from add_ons.candle_rate_of_change import add_candle_ratios
from add_ons.candle_proportion_simple import add_candle_shape_features
from add_ons.normalize_candle_seq import add_label_normalized_candles
from utils.make_step import make_step

def evaluate_model(model, val_loader, mlb, threshold=0.2, return_probs=False, return_preds=False):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            logits = model(X_batch)
            probs = torch.sigmoid(logits)
            preds = (probs >= threshold).int()
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    all_probs = np.vstack(all_probs)

    # Reporting
    print("\n📊 Validation Report (Multi-label):")
    print(classification_report(all_labels, all_preds, target_names=mlb.classes_, zero_division=0))

    print("\n🧮 Multi-label Confusion Matrices (per class):")
    mcm = multilabel_confusion_matrix(all_labels, all_preds)
    for i, cls in enumerate(mlb.classes_):
        print(f"\nClass '{cls}':")
        print(mcm[i])

    exact_match = np.all(all_preds == all_labels, axis=1).mean()
    micro_acc = (all_preds == all_labels).mean()

    print("\n✅ Exact match ratio:", exact_match)
    print("✅ Micro accuracy (per-label):", micro_acc)

    # Flexible return values
    if return_probs and return_preds:
        return exact_match, micro_acc, all_probs, all_preds
    elif return_probs:
        return exact_match, micro_acc, all_probs
    elif return_preds:
        return exact_match, micro_acc, all_preds
    else:
        return exact_match, micro_acc


def train_model(
    data_csv,
    labels_json=None,
    model_out_dir="models/saved_models",
    do_validation=True,
    seq_len=1,
    hidden_dim=10,
    num_layers=1,
    lr=0.001,
    batch_size=32,
    max_epochs=200,
    save_model=False,
    return_val_accuracy=True,
    test_mode=False,
    tune_thresholds = False,
    include_no_label = False,
    label_weighting = "none"
):
    """
    Train an LSTM classification model with labels coming from JSON (in-memory CSV).
    """

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_class_{timestamp}.pt"
    meta_out = f"{model_out_dir}/lstm_meta_class_{timestamp}.pkl"

    # --- Prepare labels ---
    if labels_json is not None:
        csv_string = json_to_csv_in_memory(labels_json)   # returns CSV string
        labels_csv = io.StringIO(csv_string)              # file-like for pandas
    else:
        raise ValueError("labels_json must be provided")

    pipeline = FeaturePipeline(
        steps=[
            make_step(add_candle_shape_features),
            # make_step(add_candle_rocp),
            # make_step(add_label_normalized_candles),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
        ],
        # norm_methods={
            # "main": {
            #     "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
            #     "upper_body_ratio": "standard", "lower_body_ratio": "standard",
            #     "upper_lower_body_ratio": "standard", "Candle_Color": "standard",
                
            # }
        #         "candle_shape": {
        #             "upper_shadow": "standard",
        #             "lower_shadow": "standard",
        #             "body": "standard",
        #             "color": "standard",
        #         }
        # },
        # window_norms={
        # "main": {"open_prop": "standard", "high_prop": "standard","low_prop": "standard", "close_prop": "standard"},},

        per_window_flags=[
        False, 
        False, 
        # True
                ]
    )
        # --- Get dataset(s) ---
    if do_validation:
        train_ds, val_ds, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=True,
            debug_sample=True,
            feature_pipeline=pipeline,
            label_weighting=label_weighting,
            include_no_label = include_no_label
        )
    else:
        full_dataset, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=False,
            debug_sample=True,
            label_weighting=label_weighting,
            include_no_label =include_no_label
        )

    # --- Model config ---
    input_dim = train_ds[0][0].shape[1] if do_validation else full_dataset[0][0].shape[1]
    num_classes = len(label_encoder.classes_)
    label_weights_tensor = torch.tensor(label_weights, dtype=torch.float32)

    model = LSTMMultiLabelClassifier(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=num_classes,
        lr=lr,
        label_weights_tensor=label_weights_tensor
    )

    init_args = {
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "num_layers": num_layers,
    "num_classes": num_classes,
    "lr": lr,

}

    model_class_info = {
        "module": model.__class__.__module__ ,
        "class": model.__class__.__name__ ,
        "init_args": init_args
    }
    # --- DataLoaders ---
    if do_validation:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=batch_size)
    else:
        train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)
        val_loader = None

    # --- Debug batch ---
    if test_mode:
        global df_seq
        df_seq = print_batch(train_loader, feature_columns, batch_idx=2)

    # --- Trainer ---
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        log_every_n_steps=10,
        fast_dev_run=test_mode,
    )

    trainer.fit(model, train_loader, val_loader)

    # --- Save model & metadata ---
    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "input_dim": input_dim,
            "hidden_dim": hidden_dim,
            "num_layers": num_layers,
            "num_classes": num_classes,
            "seq_len": seq_len,
            "lr": lr,
            "feature_columns": feature_columns,
            "label_classes": label_encoder.classes_,
            "scalers": pipeline.scalers,
            "window_scalers": pipeline.window_scalers,
            "pipeline_config": pipeline.export_config(),
            "model_class_info": model_class_info,
            "include_no_label":include_no_label
        }, meta_out)
        print(f"\n✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")

    # --- Validation accuracy ---
    val_acc_exact, val_acc_micro = None, None

    if do_validation:
        y_true_val = np.vstack([y for _, y in val_loader.dataset])

        # default eval
        val_acc_exact_default, val_acc_micro_default, y_probs, y_pred_default = evaluate_model(
            model, val_loader, label_encoder, threshold=0.5, return_probs=True, return_preds=True
        )

        if tune_thresholds:
            optimal_thresholds = tune_thresholds_nn(y_true=y_true_val, y_probs=y_probs)
            y_pred_tuned = (y_probs >= np.array(optimal_thresholds)).astype(int)

            val_acc_exact_tuned = np.all(y_pred_tuned == y_true_val, axis=1).mean()
            val_acc_micro_tuned = (y_pred_tuned == y_true_val).mean()
        else:
            val_acc_exact_tuned, val_acc_micro_tuned = val_acc_exact_default, val_acc_micro_default
            y_pred_tuned = y_pred_default   # <-- FIX: ensures it's always defined

        if return_val_accuracy:
            from sklearn.metrics import f1_score
            val_f1_macro = f1_score(y_true_val, y_pred_tuned, average="macro", zero_division=0)

            return {
                "accuracy_exact": val_acc_exact_tuned,
                "accuracy_micro": val_acc_micro_tuned,
                "f1_macro": val_f1_macro
            }


if __name__ == "__main__":
    train_model(
        data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        labels_json="/home/iatell/projects/meta-learning/data/candle_labels.json",  # JSON labels, no CSV needed on disk
        do_validation=True,
        save_model=True,
        include_no_label = True,
        label_weighting="scale_pos",
        test_mode= False
    )


## cnn fnn

In [ ]:
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from datetime import datetime
import pandas as pd
import io
import os
import numpy as np
from preprocess.multilabel_preprocess2 import preprocess_csv_multilabel
from utils.print_batch import print_batch
from utils.json_to_csv import json_to_csv_in_memory  # <-- new util
from utils.multilabel_threshold_tuning import tune_thresholds_nn
from add_ons.feature_pipeline5 import FeaturePipeline
from add_ons.drop_columns2 import drop_columns
from add_ons.candle_dif_rate_of_change_percentage2 import add_candle_rocp
from add_ons.candle_proportion import add_candle_proportions
from add_ons.candle_rate_of_change import add_candle_ratios
from add_ons.candle_proportion_simple import add_candle_shape_features
from add_ons.normalize_candle_seq import add_label_normalized_candles
from utils.make_step import make_step

import numpy as np
import torch
from sklearn.metrics import classification_report, multilabel_confusion_matrix

def evaluate_model(model, val_loader, mlb, threshold=0.2, return_probs=False, return_preds=False):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            logits = model(X_batch)
            probs = torch.sigmoid(logits)
            preds = (probs >= threshold).int()
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    all_probs = np.vstack(all_probs)

    # Reporting
    print("\n📊 Validation Report (Multi-label):")
    print(classification_report(all_labels, all_preds, target_names=mlb.classes_, zero_division=0))

    print("\n🧮 Multi-label Confusion Matrices (per class):")
    mcm = multilabel_confusion_matrix(all_labels, all_preds)
    for i, cls in enumerate(mlb.classes_):
        print(f"\nClass '{cls}':")
        print(mcm[i])

    exact_match = np.all(all_preds == all_labels, axis=1).mean()
    micro_acc = (all_preds == all_labels).mean()

    print("\n✅ Exact match ratio:", exact_match)
    print("✅ Micro accuracy (per-label):", micro_acc)

    # Flexible return values
    if return_probs and return_preds:
        return exact_match, micro_acc, all_probs, all_preds
    elif return_probs:
        return exact_match, micro_acc, all_probs
    elif return_preds:
        return exact_match, micro_acc, all_preds
    else:
        return exact_match, micro_acc



def train_model(
    data_csv,
    labels_json=None,
    model_out_dir="models/saved_models",
    do_validation=True,
    seq_len=1,
    # hidden_dim=10,
    output_channel=20,
    lr=0.001,
    batch_size=32,
    max_epochs=200,
    save_model=False,
    return_val_accuracy=True,
    test_mode=False,
    tune_thresholds = False,
    include_no_label = False,
    label_weighting = "none",
    scheduler_name = "reduce_on_plateau",
    optimizer_params={"weight_decay": 0.01},
    scheduler_params={"factor": 0.2, "patience": 3} ,
    optimizer_name= "adamw",
    dropout = 0.2
):
    """
    Train an LSTM classification model with labels coming from JSON (in-memory CSV).
    """

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_class_{timestamp}.pt"
    meta_out = f"{model_out_dir}/lstm_meta_class_{timestamp}.pkl"

    # --- Prepare labels ---
    if labels_json is not None:
        csv_string = json_to_csv_in_memory(labels_json)   # returns CSV string
        labels_csv = io.StringIO(csv_string)              # file-like for pandas
    else:
        raise ValueError("labels_json must be provided")

    pipeline = FeaturePipeline(
        steps=[
            make_step(add_candle_shape_features),
            # make_step(add_candle_rocp),
            # make_step(add_label_normalized_candles),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
        ],
        # norm_methods={
            # "main": {
            #     "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
            #     "upper_body_ratio": "standard", "lower_body_ratio": "standard",
            #     "upper_lower_body_ratio": "standard", "Candle_Color": "standard",
                
            # }
        #         "candle_shape": {
        #             "upper_shadow": "standard",
        #             "lower_shadow": "standard",
        #             "body": "standard",
        #             "color": "standard",
        #         }
        # },
        # window_norms={
        # "main": {"open_prop": "standard", "high_prop": "standard","low_prop": "standard", "close_prop": "standard"},},

        per_window_flags=[
        False, 
        False, 
        # True
                ]
    )
        # --- Get dataset(s) ---
    if do_validation:
        train_ds, val_ds, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=True,
            debug_sample=True,
            feature_pipeline=pipeline,
            label_weighting=label_weighting,
            include_no_label = include_no_label
        )
    else:
        full_dataset, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=False,
            debug_sample=True,
            label_weighting=label_weighting,
            include_no_label =include_no_label
        )

    # --- Model config ---
    input_dim = train_ds[0][0].shape[1] if do_validation else full_dataset[0][0].shape[1]
    num_classes = len(label_encoder.classes_)
    label_weights_tensor = torch.tensor(label_weights, dtype=torch.float32)

    model = candle_model(
        input_dim=input_dim,
        # hidden_dim=hidden_dim,
        num_classes=num_classes,
        output_channel=output_channel,
        lr=lr,
        label_weights_tensor=label_weights_tensor,
        scheduler_name = scheduler_name,
        optimizer_name= optimizer_name,
        optimizer_params= optimizer_params,
        scheduler_params= scheduler_params ,
        dropout= dropout
    )

    init_args = {
    "input_dim": input_dim,
    # "hidden_dim": hidden_dim,
    "output_channel": output_channel,
    "num_classes": num_classes,
    "lr": lr,
    "scheduler_name" : scheduler_name,
    "optimizer_params":optimizer_params,
    "scheduler_params":scheduler_params,
    "optimizer_name": optimizer_name,
    "dropout":dropout
}

    model_class_info = {
        "module": model.__class__.__module__ ,
        "class": model.__class__.__name__ ,
        "init_args": init_args
    }
    # --- DataLoaders ---
    if do_validation:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=batch_size)
    else:
        train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)
        val_loader = None

    # --- Debug batch ---
    if test_mode:
        global df_seq
        df_seq = print_batch(train_loader, feature_columns, batch_idx=2)

    # --- Trainer ---
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        log_every_n_steps=10,
        fast_dev_run=test_mode,
    )

    trainer.fit(model, train_loader, val_loader)

    # --- Save model & metadata ---
    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "input_dim": input_dim,
            # "hidden_dim": hidden_dim,
            "output_channel": output_channel,
            "num_classes": num_classes,
            "seq_len": seq_len,
            "lr": lr,
            "feature_columns": feature_columns,
            "label_classes": label_encoder.classes_,
            "scalers": pipeline.scalers,
            "window_scalers": pipeline.window_scalers,
            "pipeline_config": pipeline.export_config(),
            "model_class_info": model_class_info,
            "include_no_label":include_no_label
        }, meta_out)
        print(f"\n✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")

    # --- Validation accuracy ---
    val_acc_exact, val_acc_micro = None, None


    if do_validation and return_val_accuracy:
        y_true_val = np.vstack([y for _, y in val_loader.dataset])

        # Always run default evaluation to get probabilities
        val_acc_exact_default, val_acc_micro_default, y_probs, y_pred_default = evaluate_model(
            model, val_loader, label_encoder, threshold=0.5, return_probs=True, return_preds=True
        )

        # Use tuned thresholds if the option is enabled
        if tune_thresholds:
            optimal_thresholds = tune_thresholds_nn(y_true=y_true_val, y_probs=y_probs)
            y_pred_tuned = (y_probs >= np.array(optimal_thresholds)).astype(int)
        else:
            y_pred_tuned = y_pred_default

        # Calculate all final metrics based on the chosen prediction set
        from sklearn.metrics import f1_score, accuracy_score

        # Use the `y_pred_tuned` for all final metrics
        final_val_acc_exact = np.all(y_pred_tuned == y_true_val, axis=1).mean()
        final_val_acc_micro = (y_pred_tuned == y_true_val).mean()
        final_val_f1_macro = f1_score(y_true_val, y_pred_tuned, average="macro", zero_division=0)
        return {
            "val_acc_exact": float(final_val_acc_exact),
            "val_acc_micro": float(final_val_acc_micro),
            "val_f1_macro": float(final_val_f1_macro)
        }

    return {} # Return an empty dict if not returning accuracy

if __name__ == "__main__":
    train_model(
        data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        labels_json="/home/iatell/projects/meta-learning/data/candle_labels.json",  # JSON labels, no CSV needed on disk
        do_validation=True,
        save_model=False,
        include_no_label = False,
        label_weighting="scale_pos",
        test_mode= False,
        output_channel = 20
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name              | Type              | Params | Mode 
----------------------------------------------------------------
0 | feature_extractor | Sequential        | 140    | train
1 | fc                | Linear            | 126    | train
2 | criterion         | BCEWithLogitsLoss | 0      | train
----------------------------------------------------------------
266       Trainable params
0         Non-trainable params
26


=== DEBUG SAMPLE CHECK ===
Total sequences collected: 444

--- Sequence 0 ---
Original label(s): ['H']
Cleaned label(s): ['H']
Encoded: [0 1 0 0 0 0]
Feature shape: (1, 4)
First few timesteps:
 [[0.0091133  0.06722008 0.05172484 0.3       ]]



Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.



📊 Validation Report (Multi-label):
              precision    recall  f1-score   support

           +       0.76      0.95      0.85        43
           H       0.29      1.00      0.45         7
           I       0.33      1.00      0.50         2
           q       0.15      0.86      0.26         7
           s       0.58      0.92      0.71        38
           v       0.98      0.81      0.89        80

   micro avg       0.62      0.88      0.73       177
   macro avg       0.52      0.92      0.61       177
weighted avg       0.78      0.88      0.79       177
 samples avg       0.59      0.82      0.66       177


🧮 Multi-label Confusion Matrices (per class):

Class '+':
[[33 13]
 [ 2 41]]

Class 'H':
[[65 17]
 [ 0  7]]

Class 'I':
[[83  4]
 [ 0  2]]

Class 'q':
[[48 34]
 [ 1  6]]

Class 's':
[[26 25]
 [ 3 35]]

Class 'v':
[[ 8  1]
 [15 65]]

✅ Exact match ratio: 0.2696629213483146
✅ Micro accuracy (per-label): 0.7846441947565543


## full fnn

In [3]:
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from datetime import datetime
import pandas as pd
import io
import os
import numpy as np
from preprocess.multilabel_preprocess2 import preprocess_csv_multilabel
from utils.print_batch import print_batch
from utils.json_to_csv import json_to_csv_in_memory  # <-- new util
from utils.multilabel_threshold_tuning import tune_thresholds_nn
from add_ons.feature_pipeline5 import FeaturePipeline
from add_ons.drop_columns2 import drop_columns
from add_ons.candle_dif_rate_of_change_percentage2 import add_candle_rocp
from add_ons.candle_proportion import add_candle_proportions
from add_ons.candle_rate_of_change import add_candle_ratios
from add_ons.candle_proportion_simple import add_candle_shape_features
from add_ons.normalize_candle_seq import add_label_normalized_candles
from utils.make_step import make_step

import numpy as np
import torch
from sklearn.metrics import classification_report, multilabel_confusion_matrix

def evaluate_model(model, val_loader, mlb, threshold=0.2, return_probs=False, return_preds=False):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            logits = model(X_batch)
            probs = torch.sigmoid(logits)
            preds = (probs >= threshold).int()
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    all_probs = np.vstack(all_probs)

    # Reporting
    print("\n📊 Validation Report (Multi-label):")
    print(classification_report(all_labels, all_preds, target_names=mlb.classes_, zero_division=0))

    print("\n🧮 Multi-label Confusion Matrices (per class):")
    mcm = multilabel_confusion_matrix(all_labels, all_preds)
    for i, cls in enumerate(mlb.classes_):
        print(f"\nClass '{cls}':")
        print(mcm[i])

    exact_match = np.all(all_preds == all_labels, axis=1).mean()
    micro_acc = (all_preds == all_labels).mean()

    print("\n✅ Exact match ratio:", exact_match)
    print("✅ Micro accuracy (per-label):", micro_acc)

    # Flexible return values
    if return_probs and return_preds:
        return exact_match, micro_acc, all_probs, all_preds
    elif return_probs:
        return exact_match, micro_acc, all_probs
    elif return_preds:
        return exact_match, micro_acc, all_preds
    else:
        return exact_match, micro_acc


def train_model(
    data_csv,
    labels_json=None,
    model_out_dir="models/saved_models",
    do_validation=True,
    seq_len=1,
    hidden_dim=20,  # <-- NEW: Renamed output_channel to hidden_dim
    lr=0.001,
    batch_size=32,
    max_epochs=200,
    save_model=False,
    return_val_accuracy=True,
    test_mode=False,
    tune_thresholds=False,
    include_no_label=False,
    label_weighting="none",
    scheduler_name="reduce_on_plateau",
    optimizer_params={"weight_decay": 0.01},
    scheduler_params={"factor": 0.2, "patience": 3},
    optimizer_name="adamw",
    dropout=0.2
):
    """
    Train an LSTM classification model with labels coming from JSON (in-memory CSV).
    """

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_class_{timestamp}.pt"
    meta_out = f"{model_out_dir}/lstm_meta_class_{timestamp}.pkl"

    # --- Prepare labels ---
    if labels_json is not None:
        csv_string = json_to_csv_in_memory(labels_json)   # returns CSV string
        labels_csv = io.StringIO(csv_string)              # file-like for pandas
    else:
        raise ValueError("labels_json must be provided")

    pipeline = FeaturePipeline(
        steps=[
            make_step(add_candle_shape_features),
            # make_step(add_candle_rocp),
            # make_step(add_label_normalized_candles),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
        ],
        # norm_methods={
            # "main": {
            #     "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
            #     "upper_body_ratio": "standard", "lower_body_ratio": "standard",
            #     "upper_lower_body_ratio": "standard", "Candle_Color": "standard",
                
            # }
        #         "candle_shape": {
        #             "upper_shadow": "standard",
        #             "lower_shadow": "standard",
        #             "body": "standard",
        #             "color": "standard",
        #         }
        # },
        # window_norms={
        # "main": {"open_prop": "standard", "high_prop": "standard","low_prop": "standard", "close_prop": "standard"},},

        per_window_flags=[
        False, 
        False, 
        # True
                ],
    )
        # --- Get dataset(s) ---
    if do_validation:
        train_ds, val_ds, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=True,
            debug_sample=True,
            feature_pipeline=pipeline,
            label_weighting=label_weighting,
            include_no_label = include_no_label
        )
    else:
        full_dataset, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=False,
            debug_sample=True,
            label_weighting=label_weighting,
            include_no_label =include_no_label
        )

    # --- Model config ---
    input_dim = train_ds[0][0].shape[1] if do_validation else full_dataset[0][0].shape[1]
    num_classes = len(label_encoder.classes_)
    label_weights_tensor = torch.tensor(label_weights, dtype=torch.float32)

    model = candle_model(
        input_dim=input_dim,
        hidden_dim=hidden_dim,  
        num_classes=num_classes,
        lr=lr,
        label_weights_tensor=label_weights_tensor,
        scheduler_name=scheduler_name,
        optimizer_name=optimizer_name,
        optimizer_params=optimizer_params,
        scheduler_params=scheduler_params,
        dropout=dropout
    )

    init_args = {
        "input_dim": input_dim,
        "hidden_dim": hidden_dim, 
        "num_classes": num_classes,
        "lr": lr,
        "scheduler_name": scheduler_name,
        "optimizer_params": optimizer_params,
        "scheduler_params": scheduler_params,
        "optimizer_name": optimizer_name,
        "dropout": dropout
    }

    model_class_info = {
        "module": model.__class__.__module__ ,
        "class": model.__class__.__name__ ,
        "init_args": init_args
    }
    # --- DataLoaders ---
    if do_validation:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=batch_size)
    else:
        train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)
        val_loader = None

    # --- Debug batch ---
    if test_mode:
        global df_seq
        df_seq = print_batch(train_loader, feature_columns, batch_idx=2)

    # --- Trainer ---
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        log_every_n_steps=10,
        fast_dev_run=test_mode,
    )

    trainer.fit(model, train_loader, val_loader)

    # --- Save model & metadata ---
    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "input_dim": input_dim,
            "hidden_dim": hidden_dim,
            # "output_channel": output_channel,
            "num_classes": num_classes,
            "seq_len": seq_len,
            "lr": lr,
            "feature_columns": feature_columns,
            "label_classes": label_encoder.classes_,
            "scalers": pipeline.scalers,
            "window_scalers": pipeline.window_scalers,
            "pipeline_config": pipeline.export_config(),
            "model_class_info": model_class_info,
            "include_no_label":include_no_label
        }, meta_out)
        print(f"\n✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")

    # --- Validation accuracy ---
    val_acc_exact, val_acc_micro = None, None

    if do_validation and return_val_accuracy:
        y_true_val = np.vstack([y for _, y in val_loader.dataset])

        # Always run default evaluation to get probabilities
        val_acc_exact_default, val_acc_micro_default, y_probs, y_pred_default = evaluate_model(
            model, val_loader, label_encoder, threshold=0.5, return_probs=True, return_preds=True
        )

        # Use tuned thresholds if the option is enabled
        if tune_thresholds:
            optimal_thresholds = tune_thresholds_nn(y_true=y_true_val, y_probs=y_probs)
            y_pred_tuned = (y_probs >= np.array(optimal_thresholds)).astype(int)
        else:
            y_pred_tuned = y_pred_default

        # Calculate all final metrics based on the chosen prediction set
        from sklearn.metrics import f1_score, accuracy_score

        # Use the `y_pred_tuned` for all final metrics
        final_val_acc_exact = np.all(y_pred_tuned == y_true_val, axis=1).mean()
        final_val_acc_micro = (y_pred_tuned == y_true_val).mean()
        final_val_f1_macro = f1_score(y_true_val, y_pred_tuned, average="macro", zero_division=0)
        return {
            "val_acc_exact": float(final_val_acc_exact),
            "val_acc_micro": float(final_val_acc_micro),
            "val_f1_macro": float(final_val_f1_macro)
        }

    return {} # Return an empty dict if not returning accuracy



if __name__ == "__main__":
    train_model(
        data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        labels_json="/home/iatell/projects/meta-learning/data/candle_labels.json",  # JSON labels, no CSV needed on disk
        do_validation=True,
        save_model=False,
        include_no_label = False,
        label_weighting="scale_pos",
        test_mode= False,
        # output_channel = 20
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision



=== DEBUG SAMPLE CHECK ===
Total sequences collected: 444

--- Sequence 0 ---
Original label(s): ['H']
Cleaned label(s): ['H']
Encoded: [0 1 0 0 0 0]
Feature shape: (1, 4)
First few timesteps:
 [[0.0091133  0.06722008 0.05172484 0.3       ]]



2025-09-24 19:03:18.395508: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-24 19:03:18.629331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758727998.708937   17710 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758727998.732097   17710 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758727998.920936   17710 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.



📊 Validation Report (Multi-label):
              precision    recall  f1-score   support

           +       0.75      1.00      0.86        43
           H       0.43      0.86      0.57         7
           I       0.50      1.00      0.67         2
           q       0.20      1.00      0.33         7
           s       0.64      0.95      0.77        38
           v       0.97      0.95      0.96        80

   micro avg       0.70      0.96      0.81       177
   macro avg       0.58      0.96      0.69       177
weighted avg       0.79      0.96      0.85       177
 samples avg       0.70      0.93      0.78       177


🧮 Multi-label Confusion Matrices (per class):

Class '+':
[[32 14]
 [ 0 43]]

Class 'H':
[[74  8]
 [ 1  6]]

Class 'I':
[[85  2]
 [ 0  2]]

Class 'q':
[[54 28]
 [ 0  7]]

Class 's':
[[31 20]
 [ 2 36]]

Class 'v':
[[ 7  2]
 [ 4 76]]

✅ Exact match ratio: 0.4044943820224719
✅ Micro accuracy (per-label): 0.848314606741573


## fnn cnn fnn

In [72]:
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from datetime import datetime
import pandas as pd
import io
import os
import numpy as np
from preprocess.multilabel_preprocess2 import preprocess_csv_multilabel
from utils.print_batch import print_batch
from utils.json_to_csv import json_to_csv_in_memory  # <-- new util
from utils.multilabel_threshold_tuning import tune_thresholds_nn
from add_ons.feature_pipeline5 import FeaturePipeline
from add_ons.drop_columns2 import drop_columns
from add_ons.candle_dif_rate_of_change_percentage2 import add_candle_rocp
from add_ons.candle_proportion import add_candle_proportions
from add_ons.candle_rate_of_change import add_candle_ratios
from add_ons.candle_proportion_simple import add_candle_shape_features
from add_ons.normalize_candle_seq import add_label_normalized_candles
from utils.make_step import make_step

import numpy as np
import torch
from sklearn.metrics import classification_report, multilabel_confusion_matrix

def evaluate_model(model, val_loader, mlb, threshold=0.2, return_probs=False, return_preds=False):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            logits = model(X_batch)
            probs = torch.sigmoid(logits)
            preds = (probs >= threshold).int()
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    all_probs = np.vstack(all_probs)

    # Reporting
    print("\n📊 Validation Report (Multi-label):")
    print(classification_report(all_labels, all_preds, target_names=mlb.classes_, zero_division=0))

    print("\n🧮 Multi-label Confusion Matrices (per class):")
    mcm = multilabel_confusion_matrix(all_labels, all_preds)
    for i, cls in enumerate(mlb.classes_):
        print(f"\nClass '{cls}':")
        print(mcm[i])

    exact_match = np.all(all_preds == all_labels, axis=1).mean()
    micro_acc = (all_preds == all_labels).mean()

    print("\n✅ Exact match ratio:", exact_match)
    print("✅ Micro accuracy (per-label):", micro_acc)

    # Flexible return values
    if return_probs and return_preds:
        return exact_match, micro_acc, all_probs, all_preds
    elif return_probs:
        return exact_match, micro_acc, all_probs
    elif return_preds:
        return exact_match, micro_acc, all_preds
    else:
        return exact_match, micro_acc



def train_model(
    data_csv,
    labels_json=None,
    model_out_dir="models/saved_models",
    do_validation=True,
    seq_len=1,
    fnn_hidden_dim=20,  # <-- NEW: Hidden dimension of the FNN layer
    output_channel=20,  # <-- NEW: Output channels of the CNN layer
    lr=0.001,
    batch_size=32,
    max_epochs=200,
    save_model=False,
    return_val_accuracy=True,
    test_mode=False,
    tune_thresholds=False,
    include_no_label=False,
    label_weighting="none",
    scheduler_name="reduce_on_plateau",
    optimizer_params={"weight_decay": 0.01},
    scheduler_params={"factor": 0.2, "patience": 3},
    optimizer_name="adamw",
    dropout=0.2
):
    """
    Train an LSTM classification model with labels coming from JSON (in-memory CSV).
    """

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_class_{timestamp}.pt"
    meta_out = f"{model_out_dir}/lstm_meta_class_{timestamp}.pkl"

    # --- Prepare labels ---
    if labels_json is not None:
        csv_string = json_to_csv_in_memory(labels_json)   # returns CSV string
        labels_csv = io.StringIO(csv_string)              # file-like for pandas
    else:
        raise ValueError("labels_json must be provided")

    pipeline = FeaturePipeline(
        steps=[
            make_step(add_candle_shape_features),
            # make_step(add_candle_rocp),
            # make_step(add_label_normalized_candles),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
        ],
        # norm_methods={
            # "main": {
            #     "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
            #     "upper_body_ratio": "standard", "lower_body_ratio": "standard",
            #     "upper_lower_body_ratio": "standard", "Candle_Color": "standard",
                
            # }
        #         "candle_shape": {
        #             "upper_shadow": "standard",
        #             "lower_shadow": "standard",
        #             "body": "standard",
        #             "color": "standard",
        #         }
        # },
        # window_norms={
        # "main": {"open_prop": "standard", "high_prop": "standard","low_prop": "standard", "close_prop": "standard"},},

        per_window_flags=[
        False, 
        False, 
        # True
                ]
    )
        # --- Get dataset(s) ---
    if do_validation:
        train_ds, val_ds, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=True,
            debug_sample=True,
            feature_pipeline=pipeline,
            label_weighting=label_weighting,
            include_no_label = include_no_label
        )
    else:
        full_dataset, df, feature_columns, label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=False,
            debug_sample=True,
            label_weighting=label_weighting,
            include_no_label =include_no_label
        )

    # --- Model config ---
    input_dim = train_ds[0][0].shape[1] if do_validation else full_dataset[0][0].shape[1]
    num_classes = len(label_encoder.classes_)
    label_weights_tensor = torch.tensor(label_weights, dtype=torch.float32)

    model = candle_model(
        input_dim=input_dim,
        fnn_hidden_dim=fnn_hidden_dim,  # <-- UPDATED
        output_channel=output_channel,  # <-- UPDATED
        num_classes=num_classes,
        lr=lr,
        label_weights_tensor=label_weights_tensor,
        scheduler_name=scheduler_name,
        optimizer_name=optimizer_name,
        optimizer_params=optimizer_params,
        scheduler_params=scheduler_params,
        dropout=dropout
    )

    init_args = {
        "input_dim": input_dim,
        "fnn_hidden_dim": fnn_hidden_dim,  # <-- UPDATED
        "output_channel": output_channel,  # <-- UPDATED
        "num_classes": num_classes,
        "lr": lr,
        "scheduler_name": scheduler_name,
        "optimizer_params": optimizer_params,
        "scheduler_params": scheduler_params,
        "optimizer_name": optimizer_name,
        "dropout": dropout
    }

    model_class_info = {
        "module": model.__class__.__module__ ,
        "class": model.__class__.__name__ ,
        "init_args": init_args
    }
    # --- DataLoaders ---
    if do_validation:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=batch_size)
    else:
        train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)
        val_loader = None

    # --- Debug batch ---
    if test_mode:
        global df_seq
        df_seq = print_batch(train_loader, feature_columns, batch_idx=2)

    # --- Trainer ---
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        log_every_n_steps=10,
        fast_dev_run=test_mode,
    )

    trainer.fit(model, train_loader, val_loader)

    # --- Save model & metadata ---
    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "input_dim": input_dim,
            # "hidden_dim": hidden_dim,
            "output_channel": output_channel,
            "num_classes": num_classes,
            "seq_len": seq_len,
            "lr": lr,
            "feature_columns": feature_columns,
            "label_classes": label_encoder.classes_,
            "scalers": pipeline.scalers,
            "window_scalers": pipeline.window_scalers,
            "pipeline_config": pipeline.export_config(),
            "model_class_info": model_class_info,
            "include_no_label":include_no_label
        }, meta_out)
        print(f"\n✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")

    # --- Validation accuracy ---
    val_acc_exact, val_acc_micro = None, None


    if do_validation and return_val_accuracy:
        y_true_val = np.vstack([y for _, y in val_loader.dataset])

        # Always run default evaluation to get probabilities
        val_acc_exact_default, val_acc_micro_default, y_probs, y_pred_default = evaluate_model(
            model, val_loader, label_encoder, threshold=0.5, return_probs=True, return_preds=True
        )

        # Use tuned thresholds if the option is enabled
        if tune_thresholds:
            optimal_thresholds = tune_thresholds_nn(y_true=y_true_val, y_probs=y_probs)
            y_pred_tuned = (y_probs >= np.array(optimal_thresholds)).astype(int)
        else:
            y_pred_tuned = y_pred_default

        # Calculate all final metrics based on the chosen prediction set
        from sklearn.metrics import f1_score, accuracy_score

        # Use the `y_pred_tuned` for all final metrics
        final_val_acc_exact = np.all(y_pred_tuned == y_true_val, axis=1).mean()
        final_val_acc_micro = (y_pred_tuned == y_true_val).mean()
        final_val_f1_macro = f1_score(y_true_val, y_pred_tuned, average="macro", zero_division=0)
        return {
            "val_acc_exact": float(final_val_acc_exact),
            "val_acc_micro": float(final_val_acc_micro),
            "val_f1_macro": float(final_val_f1_macro)
        }

    return {} # Return an empty dict if not returning accuracy


if __name__ == "__main__":
    train_model(
        data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        labels_json="/home/iatell/projects/meta-learning/data/candle_labels.json",  # JSON labels, no CSV needed on disk
        do_validation=True,
        save_model=False,
        include_no_label = False,
        label_weighting="scale_pos",
        test_mode= False,
        output_channel = 20
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params | Mode 
------------------------------------------------------------
0 | fnn_layer     | Sequential        | 140    | train
1 | cnn_extractor | Sequential        | 460    | train
2 | fc            | Linear            | 126    | train
3 | criterion     | BCEWithLogitsLoss | 0      | train
------------------------------------------------------------
726       Trainable params
0         Non-trainable params
726       Total params
0.003     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode



=== DEBUG SAMPLE CHECK ===
Total sequences collected: 444

--- Sequence 0 ---
Original label(s): ['H']
Cleaned label(s): ['H']
Encoded: [0 1 0 0 0 0]
Feature shape: (1, 4)
First few timesteps:
 [[0.0091133  0.06722008 0.05172484 0.3       ]]



Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.



📊 Validation Report (Multi-label):
              precision    recall  f1-score   support

           +       0.59      1.00      0.74        43
           H       0.40      0.57      0.47         7
           I       0.50      1.00      0.67         2
           q       0.17      1.00      0.29         7
           s       0.49      0.95      0.65        38
           v       0.96      1.00      0.98        80

   micro avg       0.61      0.97      0.75       177
   macro avg       0.52      0.92      0.63       177
weighted avg       0.71      0.97      0.80       177
 samples avg       0.60      0.96      0.71       177


🧮 Multi-label Confusion Matrices (per class):

Class '+':
[[16 30]
 [ 0 43]]

Class 'H':
[[76  6]
 [ 3  4]]

Class 'I':
[[85  2]
 [ 0  2]]

Class 'q':
[[48 34]
 [ 0  7]]

Class 's':
[[14 37]
 [ 2 36]]

Class 'v':
[[ 6  3]
 [ 0 80]]

✅ Exact match ratio: 0.23595505617977527
✅ Micro accuracy (per-label): 0.7808988764044944


# XGboost

In [1]:
import joblib
from datetime import datetime
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, multilabel_confusion_matrix, f1_score
import os
import io
import numpy as np
from preprocess.multilabel_preprocess2 import preprocess_csv_multilabel
from utils.json_to_csv import json_to_csv_in_memory
from utils.multilabel_threshold_tuning import tune_thresholds
from add_ons.feature_pipeline5 import FeaturePipeline
from utils.make_step import make_step
from add_ons.drop_columns2 import drop_columns
from add_ons.candle_dif_rate_of_change_percentage2 import add_candle_rocp
from add_ons.candle_proportion import add_candle_proportions
from add_ons.candle_rate_of_change import add_candle_ratios
from add_ons.candle_proportion_simple import add_candle_shape_features
from add_ons.normalize_candle_seq import add_label_normalized_candles

def evaluate_multilabel_model(model, X_val, y_val, mlb, thresholds=None):
    """
    Evaluate a multi-label XGBoost model and print metrics.
    Optionally apply per-label thresholds.
    """
    # Predict probabilities per label
    y_probs = np.column_stack([est.predict_proba(X_val)[:, 1] for est in model.estimators_])

    # Apply thresholds
    if thresholds is None:
        thresholds = [0.5] * y_val.shape[1]
    y_pred = np.zeros_like(y_val)
    for i, t in enumerate(thresholds):
        y_pred[:, i] = (y_probs[:, i] >= t).astype(int)

    print("\n📊 Validation Report (Multi-label):")
    print(classification_report(y_val, y_pred, target_names=mlb.classes_, zero_division=0))

    print("\n🧮 Multi-label Confusion Matrices (per class):")
    mcm = multilabel_confusion_matrix(y_val, y_pred)
    for i, cls in enumerate(mlb.classes_):
        print(f"\nClass '{cls}':")
        print(mcm[i])

    exact_match = np.all(y_pred == y_val, axis=1).mean()
    print("\nExact match ratio:", exact_match)

    micro_acc = (y_pred == y_val).mean()
    print("Micro accuracy (per-label):", micro_acc)

    return exact_match, micro_acc, y_probs


def train_model_xgb_multilabel(
    data_csv,
    labels_json,
    model_out_dir="models/saved_models",
    do_validation=True,
    seq_len=1,
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    save_model=False,
    return_val_accuracy=True,
    label_weighting="none",  # "none", dict, or "scale_pos"
    threshold_tuning = False,
    include_no_label = False,
):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/xgb_model_multilabel_{timestamp}.pkl"
    meta_out = f"{model_out_dir}/xgb_meta_multilabel_{timestamp}.pkl"

    csv_string = json_to_csv_in_memory(labels_json)
    labels_csv = io.StringIO(csv_string)

    pipeline = FeaturePipeline(
        steps=[
            make_step(add_candle_shape_features),
            # make_step(add_candle_rocp),
            # make_step(add_label_normalized_candles),
            make_step(drop_columns, cols_to_drop=["open","high","low","close","volume"]),
        ],
        # norm_methods={
            # "main": {
            #     "upper_shadow": "robust", "body": "standard", "lower_shadow": "standard",
            #     "upper_body_ratio": "standard", "lower_body_ratio": "standard",
            #     "upper_lower_body_ratio": "standard", "Candle_Color": "standard",
                
            # }
        #         "candle_shape": {
        #             "upper_shadow": "standard",
        #             "lower_shadow": "standard",
        #             "body": "standard",
        #             "color": "standard",
        #         }
        # },flatten
        # window_norms={
        # "main": {"open_prop": "standard", "high_prop": "standard","low_prop": "standard", "close_prop": "standard"},},
        # transformations={"mode": "rocket", "num_kernels": 500, "normalise": True, "strategy": "forward_fill"},
        transformations={"mode": "flatten"},
        per_window_flags=[
        False, 
        False, 
        # True
                ]
    )
    if do_validation:
        X_train,y_train, X_val, y_val, df, feature_columns,label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=True,
            for_xgboost=True,
            debug_sample=[0, 1],
            label_weighting=label_weighting,
            feature_pipeline=pipeline,
            include_no_label = include_no_label
        )
    else:
        X_train, y_train, df, feature_columns,label_encoder, label_weights = preprocess_csv_multilabel(
            data_csv, labels_csv,
            n_candles=seq_len,
            val_split=False,
            for_xgboost=True,
            label_weighting=label_weighting,
            feature_pipeline=pipeline,
            include_no_label=include_no_label
        )
        X_val, y_val = None, None

    xgb_models = []
    for w in label_weights:
        xgb_model = xgb.XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            eval_metric='logloss',
            scale_pos_weight=w,
        )
        xgb_models.append(xgb_model)

    model = MultiOutputClassifier(xgb_models[0], n_jobs=-1)
    model.estimators_ = xgb_models
    model.fit(X_train, y_train)

    # Tune thresholds if validation set exists
    optimal_thresholds = None
    val_acc_exact, val_acc_micro = None, None
    if do_validation:
        # --- Step 1: Predict probabilities once ---
        y_probs = np.column_stack([est.predict_proba(X_val)[:, 1] for est in model.estimators_])

        # --- Step 2: Evaluate with default threshold 0.5 ---
        val_acc_exact_default, val_acc_micro_default, _ = evaluate_multilabel_model(
            model, X_val, y_val, label_encoder, thresholds=[0.5]*y_val.shape[1]
        )
        if threshold_tuning:
        # --- Step 3: Tune optimal thresholds per label ---
            optimal_thresholds = tune_thresholds(y_val, y_probs)
            print("\n📌 Optimal thresholds per label:", dict(zip(label_encoder.classes_, optimal_thresholds)))

            # --- Step 4: Evaluate with tuned thresholds ---
            val_acc_exact_tuned, val_acc_micro_tuned, _ = evaluate_multilabel_model(
                model, X_val, y_val, label_encoder, thresholds=optimal_thresholds
            )
    if return_val_accuracy:
        return {
            "exact_match": val_acc_exact_default,
            "micro_accuracy": val_acc_micro_default,
            "label_weights": label_weights,
            "optimal_thresholds": optimal_thresholds
        }



    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        joblib.dump(model, model_out)
        joblib.dump({
            'seq_len': seq_len,
            'label_classes': label_encoder.classes_,
            'optimal_thresholds': optimal_thresholds,
            "feature_columns": feature_columns,
            "scalers": pipeline.scalers,
            "window_scalers": pipeline.window_scalers,
            "pipeline_config": pipeline.export_config(),
            "include_no_label":include_no_label
        }, meta_out)
        print(f"✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")


if __name__ == "__main__":
    train_model_xgb_multilabel(
        data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
        labels_json="/home/iatell/projects/meta-learning/data/candle_labels.json",
        do_validation=True,
        label_weighting="scale_pos"# "none", dict, or "scale_pos"
        ,include_no_label= True,
        save_model = False

    )



=== DEBUG SAMPLE CHECK ===
Total sequences collected: 1603

--- Sequence 0 ---
Original label(s): ['no_label']
Cleaned label(s): ['no_label']
Encoded: [0 0 0 1 0 0 0]
Feature shape: (1, 4)
First few timesteps:
 [[0.05440368 0.03677583 0.08810496 0.7       ]]

--- Sequence 1 ---
Original label(s): ['no_label']
Cleaned label(s): ['no_label']
Encoded: [0 0 0 1 0 0 0]
Feature shape: (1, 4)
First few timesteps:
 [[0.02600957 0.0367597  0.01538321 0.7       ]]


📊 Validation Report (Multi-label):
              precision    recall  f1-score   support

           +       0.94      0.96      0.95        51
           H       0.67      0.29      0.40         7
           I       0.00      0.00      0.00         3
    no_label       0.92      1.00      0.96       231
           q       0.00      0.00      0.00         3
           s       0.76      0.84      0.80        44
           v       0.96      0.94      0.95        81

   micro avg       0.90      0.94      0.92       420
   macro avg   

# server

In [ ]:
import sys
import glob
import joblib
import torch
import os
import numpy as np
import pandas as pd
from pathlib import Path
from flask import Flask, request, jsonify, render_template

# ---------------- Imports ----------------
from servers.pre_process.multi_reg_dif_seq2 import (
    ServerPreprocess, build_pipeline_from_config
)
from models.LSTM.lstm_multi_label import LSTMMultiLabelClassifier  # adjust path/class name


# ---------------- Flask app ----------------
app = Flask(__name__)

# ---------------- Load model + meta ----------------
meta_files = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_meta_class_*.pkl")
state_files = glob.glob("/home/iatell/projects/meta-learning/play_grounds/models/saved_models/lstm_model_class_*.pt")

if not meta_files or not state_files:
    raise FileNotFoundError("No multilabel model/meta found!")

meta_path = max(meta_files, key=os.path.getmtime)
state_path = max(state_files, key=os.path.getmtime)

print("Using meta:", meta_path)
print("Using state:", state_path)

meta = joblib.load(meta_path)
FEATURES = meta["feature_columns"]
LABEL_CLASSES = meta["label_classes"]
print("Features:", FEATURES)
print("Label classes:", LABEL_CLASSES)

# Init model
model_cls_info = meta["model_class_info"]
init_args = model_cls_info["init_args"]
model = LSTMMultiLabelClassifier.load_from_checkpoint(state_path, **init_args)
model.eval()

# ---------------- Pipeline ----------------
pipeline = build_pipeline_from_config(meta["pipeline_config"])
pipeline.scalers = meta["scalers"]
pipeline.window_scalers = meta["window_scalers"]

preproc = ServerPreprocess(feature_pipeline=pipeline)

# ---------------- Load dataset ----------------
DATA_FILE = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"  # change path as needed
df = pd.read_csv(DATA_FILE, parse_dates=["timestamp"])

# ---------------- Routes ----------------
@app.route("/")
def home():
    return render_template("multi_label.html")

@app.route("/get_and_add_data")
def get_and_add_data():
    # Convert timestamp column to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')  # or unit='ms' if needed
    # print("[DEBUG] df['timestamp'] head:", df['timestamp'].head())

    # Reindex to daily frequency and forward-fill missing
    dense = df.set_index('timestamp').asfreq('D').ffill()
    # print("[DEBUG] dense head:", dense.head())

    initial_seq_len = 21
    next_idx = request.args.get("idx", type=int)
    # print(f"[DEBUG] next_idx from request: {next_idx}")

    if next_idx is None:
        # --- First load ---
        if len(preproc.dataset) == 0:
            # print("[DEBUG] Adding initial candles to preproc.dataset")
            for _, row in dense.iloc[:initial_seq_len].iterrows():
                preproc.add_candle(row)

        candles = []
        for _, row in dense.iloc[:initial_seq_len].iterrows():
            # Use row.name which is the timestamp index
            time_val = int(row.name.timestamp())
            # print(f"[DEBUG] Candle time: {time_val}, high: {row.high}")
            candles.append({
                "time": time_val,
                "open": float(row.open),
                "high": float(row.high),
                "low": float(row.low),
                "close": float(row.close),
            })

        return jsonify({
            "initial_seq_len": initial_seq_len,
            "next_idx": initial_seq_len,
            "candles": candles
        })

    else:
        # --- Step forward ---
        if next_idx >= len(dense):
            # print("[DEBUG] End of data reached")
            return jsonify({"error": "End of data"}), 404

        row = dense.iloc[next_idx]
        time_val = int(row.name.timestamp())
        # print(f"[DEBUG] Step candle time: {time_val}, high: {row.high}")

        candle = {
            "time": time_val,
            "open": float(row.open),
            "high": float(row.high),
            "low": float(row.low),
            "close": float(row.close),
        }

        preproc.add_candle(row)
        # print(f"[DEBUG] Added candle to preproc, next_idx will be {next_idx + 1}")

        return jsonify({
            "next_idx": next_idx + 1,
            "candle": candle
        })


@app.route("/predict", methods=["POST"])
def predict():
    # Use seq_len from meta (training config)
    seq_len = meta.get("seq_len")
    if not seq_len or not isinstance(seq_len, int):
        return jsonify({"error": "Server meta is missing a valid 'seq_len'"}), 500

    # Build sequence from file / preprocessor
    try:
        seq_dict = preproc.prepare_seq(seq_len)  
    except ValueError as e:
        return jsonify({"error": str(e)}), 400

    # Convert dict of DataFrames → tensors
    dict_x = {k: torch.from_numpy(v.values.astype(np.float32)).unsqueeze(0)
              for k, v in seq_dict.items()}
    lengths = torch.tensor([seq_len], dtype=torch.long)
    with torch.no_grad():
        y_probs = model(dict_x["main"]).sigmoid().cpu().numpy()[0]

    # Decode labels above threshold
    threshold = 0.5
    predicted_labels = [LABEL_CLASSES[i] for i, p in enumerate(y_probs) if p >= threshold]

    # Get last candle time directly (it's already an int timestamp)
    return jsonify({
        "raw_probs": dict(zip(LABEL_CLASSES, y_probs.tolist())),
        "predicted_labels": predicted_labels
    })


# ---------------- Run ----------------
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


# Tuning neural network

##  lstm

In [ ]:
import os
import torch
from datetime import datetime

from ray import tune, air
from ray.tune.schedulers import ASHAScheduler

from utils.flatten_config import flatten_config
from utils.resoure_usage import resource_usage

# --- Data paths ---
data_csv = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"
labels_json = "/home/iatell/projects/meta-learning/data/candle_labels.json"


def train_model_tune(config):
    """
    Single Ray Tune trial.
    """
    resource_usage()  # Show hardware usage
    
    # --- Fixed arguments ---
    fixed_args = dict(
        data_csv=data_csv,
        labels_json=labels_json,
        do_validation=True,
        model_out_dir="models/tuned",
        return_val_accuracy=True,
        save_model=False,
    )
    
    # --- Build training args ---
    train_args = fixed_args.copy()

    # Model hyperparams
    train_args["seq_len"] = config["seq_len"]
    train_args["hidden_dim"] = config["hidden_dim"]

    # Training hyperparams
    train_args["lr"] = config["lr"]
    train_args["batch_size"] = config["batch_size"]
    train_args["max_epochs"] = config["max_epochs"]
    train_args["dropout"] = config["dropout"]

    # Optimizer
    train_args["optimizer_name"] = config["optimizer_name"]
    if train_args["optimizer_name"] in ["adamw", "adam"]:
        train_args["optimizer_params"] = {"weight_decay": config["weight_decay"]}
    else:
        train_args["optimizer_params"] = {}

    # Scheduler
    train_args["scheduler_name"] = config["scheduler_name"]
    if train_args["scheduler_name"] == "reduce_on_plateau":
        train_args["scheduler_params"] = {
            "factor": config["scheduler_params"]["factor"],
            "patience": config["scheduler_params"]["patience"]
        }
    elif train_args["scheduler_name"] == "cosine":
        train_args["scheduler_params"] = {
            "T_max": config["scheduler_params"]["T_max"],
            "eta_min": config["scheduler_params"]["eta_min"]
        }
    else:
        train_args["scheduler_params"] = {}

    # --- Run training ---
    metrics = train_model(**train_args)

    # Report back to Ray Tune
    tune.report(metrics)


def run_tuning(save_model=False):
    """Hyperparameter tuning for CNN-LSTM with Ray Tune."""
    
    search_space = {
        # Model
        "seq_len": tune.choice([1, 5, 10, 20]),
        "hidden_dim": tune.choice([32, 64, 128, 256]),
        "dropout": tune.uniform(0.1, 0.5),

        # Training
        "lr": tune.loguniform(1e-4, 1e-3),
        "batch_size": tune.choice([32, 64, 128]),
        "max_epochs": tune.choice([50, 100, 150]),

        # Optimizer
        "optimizer_name": tune.choice(["adamw", "adam"]),
        "weight_decay": tune.loguniform(1e-5, 1e-2),

        # Scheduler
        "scheduler_name": tune.choice(["reduce_on_plateau", "cosine", "none"]),
        "scheduler_params": {
            "factor": tune.uniform(0.1, 0.5),
            "patience": tune.choice([3, 5, 10]),
            "T_max": tune.choice([10, 20, 50]),
            "eta_min": tune.loguniform(1e-6, 1e-4),
        },
    }

    scheduler = ASHAScheduler(
        metric="f1_macro",
        mode="max",
        grace_period=5,
        reduction_factor=2,
    )

    # --- Add timestamp to run name ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"cnn_lstm_tuning_{timestamp}"

    tuner = tune.Tuner(
        tune.with_resources(train_model_tune, {"cpu": 1, "gpu": 1}),
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=10,
        ),
        run_config=air.RunConfig(
            name=run_name,
            storage_path="/home/iatell/projects/meta-learning/tune_logs",
        ),
    )

    results = tuner.fit()

    # --- Best trial ---
    best_result = results.get_best_result(metric="f1_macro", mode="max")
    print("\n🏆 Best Config:", best_result.config)
    print(f"Best F1 Macro: {best_result.metrics['f1_macro']:.4f}")

    # --- Retrain with best config ---
    if save_model:
        print("\n🔁 Retraining best model on full dataset for saving...")
        retrain_args = {
            "data_csv": data_csv,
            "labels_json": labels_json,
            "do_validation": False,
            "model_out_dir": "models/saved_models",
            "save_model": True,
            "seq_len": best_result.config.get("seq_len", 1),
            "hidden_dim": best_result.config["hidden_dim"],
            "lr": best_result.config["lr"],
            "batch_size": best_result.config["batch_size"],
            "max_epochs": best_result.config["max_epochs"],
            "dropout": best_result.config["dropout"],
            "optimizer_name": best_result.config["optimizer_name"],
            "scheduler_name": best_result.config["scheduler_name"],
            "optimizer_params": {"weight_decay": best_result.config["weight_decay"]},
            "scheduler_params": best_result.config["scheduler_params"],
        }
        train_model(**retrain_args)


if __name__ == "__main__":
    run_tuning(save_model=True)


## full fnn

In [49]:
import os
import torch
from utils.flatten_config import flatten_config
from utils.resoure_usage import resource_usage
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray import air

import io
from datetime import datetime
import torch
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.air import session as air
from ray.air import RunConfig
from ray import air
from ray.tune import TuneConfig
import os
from datetime import datetime

data_csv = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"
labels_json = "/home/iatell/projects/meta-learning/data/candle_labels.json"

def train_model_tune(config):
    """
    Single Ray Tune trial.
    """
    fixed_args = dict(
        data_csv=data_csv,
        labels_json=labels_json,
        do_validation=True,
        model_out_dir="models/tuned",
        return_val_accuracy=True,
        save_model=False,
    )
    
    # Start with the fixed arguments
    train_args = fixed_args.copy()
    
    # Manually add the top-level hyperparameters
    train_args['lr'] = config['lr']
    train_args['batch_size'] = config['batch_size']
    train_args['max_epochs'] = config['max_epochs']
    train_args['dropout'] = config['dropout']
    
    # 🌟 Conditional Logic for Optimizer and its params
    train_args['optimizer_name'] = config['optimizer_name']
    if train_args['optimizer_name'] in ['adamw', 'adam']:
        # Pass the weight_decay key and value directly
        train_args['optimizer_params'] = {'weight_decay': config['weight_decay']}
    else:
        # For other optimizers (e.g., SGD), weight decay might not be needed or handled differently
        train_args['optimizer_params'] = {}

    # 🌟 Conditional Logic for Scheduler
    train_args['scheduler_name'] = config['scheduler_name']
    if train_args['scheduler_name'] == 'reduce_on_plateau':
        # Pass the scheduler params as a nested dictionary
        train_args['scheduler_params'] = {
            'factor': config['scheduler_params']['factor'],
            'patience': config['scheduler_params']['patience']
        }
    else:
        # If no scheduler is chosen, pass an empty dictionary
        train_args['scheduler_params'] = {}
        
    # Manually add model-specific parameters
    if 'hidden_dim' in config:
        train_args['hidden_dim'] = config['hidden_dim']
    elif 'fnn_hidden_dim' in config and 'output_channel' in config:
        train_args['fnn_hidden_dim'] = config['fnn_hidden_dim']
        train_args['output_channel'] = config['output_channel']
    
    metrics = train_model(**train_args)
    tune.report(metrics)

def run_tuning(save_model=False):
    """Hyperparameter tuning for FNN model with Ray Tune."""
    search_space = {
        "hidden_dim": tune.choice([32, 64, 128, 256, 512]),
        "dropout": tune.uniform(0.1, 0.5),
        "lr": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "max_epochs": tune.choice([100, 150, 200]),
        
        # 🌟 Define optimizer and scheduler parameters at the top level
        "optimizer_name": tune.choice(["adamw", "adam"]),
        "weight_decay": tune.loguniform(1e-5, 1e-2), # <-- NEW: weight_decay is now at the top-level
        
        "scheduler_name": tune.choice(["reduce_on_plateau", "none"]),
        "scheduler_params": {
            "factor": tune.uniform(0.1, 0.5),
            "patience": tune.choice([3, 5, 10]),
        },
    }

    scheduler = ASHAScheduler(
        metric="val_acc_exact",
        mode="max",
        grace_period=10,
        reduction_factor=2
    )
    # --- Add timestamp to run name ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"fullfnn_multilabel_tuning_{timestamp}"
    
    tuner = tune.Tuner(
        tune.with_resources(train_model_tune, {"cpu": 1, "gpu": 1}),
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=10
        ),
        run_config=air.RunConfig(
            name=run_name,
            storage_path="/home/iatell/projects/meta-learning/tune_logs",
        ),
    )

    results = tuner.fit()
    
    best_result = results.get_best_result(metric="val_acc_exact", mode="max")
    print("\n🏆 Best Config:", best_result.config)
    print(f"Best Macro F1 Score: {best_result.metrics['val_acc_exact']:.4f}")

    if save_model:
        print("\n🔁 Retraining best model on full dataset for saving...")
        
        # 🛠️ CORRECTED: Create a clean dictionary from the best config
        retrain_args = {
            "data_csv": data_csv,
            "labels_json": labels_json,
            "do_validation": False,
            "model_out_dir": "models/saved_models",
            "save_model": True,
            "seq_len": best_result.config.get("seq_len", 1), # Example, if you add this to your search space
            "hidden_dim": best_result.config["hidden_dim"],
            "lr": best_result.config["lr"],
            "batch_size": best_result.config["batch_size"],
            "max_epochs": best_result.config["max_epochs"],
            "dropout": best_result.config["dropout"],
            "optimizer_name": best_result.config["optimizer_name"],
            "scheduler_name": best_result.config["scheduler_name"],
            
            # Manually construct the nested parameter dictionaries
            "optimizer_params": {
                'weight_decay': best_result.config['weight_decay']
            },
            "scheduler_params": best_result.config['scheduler_params']
        }

        train_model(**retrain_args)

if __name__ == "__main__":
    run_tuning(save_model=False)

2025-09-22 21:01:06,349	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/iatell/projects/meta-learning/tune_logs/fullfnn_multilabel_tuning_20250922_205929' in 0.0034s.
2025-09-22 21:01:06,354	INFO tune.py:1041 -- Total run time: 96.43 seconds (96.41 seconds for the tuning loop).



🏆 Best Config: {'hidden_dim': 512, 'dropout': 0.18761731714766047, 'lr': 0.008693076055154599, 'batch_size': 128, 'max_epochs': 150, 'optimizer_name': 'adamw', 'weight_decay': 0.0009739052977471559, 'scheduler_name': 'reduce_on_plateau', 'scheduler_params': {'factor': 0.3714526069181563, 'patience': 10}}
Best Macro F1 Score: 0.6292

🔁 Retraining best model on full dataset for saving...


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | fcn       | Sequential        | 7.2 K  | train
1 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)
8         Modules in train mode
0         Mo


=== DEBUG SAMPLE CHECK ===
Total sequences collected: 444

--- Sequence 0 ---
Original label(s): ['H']
Cleaned label(s): ['H']
Encoded: [0 1 0 0 0 0]
Feature shape: (1, 5)
First few timesteps:
 [[14210.    14339.5   12569.2   13474.99  17017.633]]



/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

MisconfigurationException: ReduceLROnPlateau conditioned on metric val_loss which is not available. Available metrics are: ['train_loss', 'train_loss_step', 'train_loss_epoch']. Condition can be set using `monitor` key in lr scheduler dict

## fnn cnn fnn

In [73]:
import os
import torch
from utils.resoure_usage import resource_usage
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray import air
from datetime import datetime

data_csv = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"
labels_json = "/home/iatell/projects/meta-learning/data/candle_labels.json"

def train_model_tune(config):
    """
    Single Ray Tune trial.
    """
    # resource_usage()  # Show current hardware usage if needed

    # Fixed args (constants)
    fixed_args = dict(
        data_csv=data_csv,
        labels_json=labels_json,
        do_validation=True,
        model_out_dir="models/tuned",
        return_val_accuracy=True,
        save_model=False,
    )

    # Start with fixed args
    train_args = fixed_args.copy()

    # 🌟 Top-level hyperparameters
    train_args["dropout"] = config["dropout"]
    train_args["lr"] = config["lr"]
    train_args["batch_size"] = config["batch_size"]
    train_args["max_epochs"] = config["max_epochs"]

    # 🌟 Optimizer
    train_args["optimizer_name"] = config["optimizer_name"]
    train_args["optimizer_params"] = {"weight_decay": config["optimizer_params"]["weight_decay"]}

    # 🌟 Scheduler
    train_args["scheduler_name"] = config["scheduler_name"]
    if train_args["scheduler_name"] == "reduce_on_plateau":
        train_args["scheduler_params"] = {
            "factor": config["scheduler_params"]["factor"],
            "patience": config["scheduler_params"]["patience"],
        }
    else:
        train_args["scheduler_params"] = {}

    # 🌟 Model-specific (fnn+cnn hybrid)
    train_args["fnn_hidden_dim"] = config["fnn_hidden_dim"]
    train_args["output_channel"] = config["output_channel"]

    # Train
    metrics = train_model(**train_args)
    tune.report(metrics)

def run_tuning(save_model=False):
    """Hyperparameter tuning for FNN-CNN hybrid model with Ray Tune."""

    search_space = {
        # Model hyperparameters
        "fnn_hidden_dim": tune.choice([32, 64, 128, 256]),
        "output_channel": tune.choice([16, 32, 64, 128]),
        "dropout": tune.uniform(0.1, 0.5),

        # Optimizer / training params
        "lr": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([32, 64, 128]),
        "max_epochs": tune.choice([100, 150, 200]),

        # Optimizer params
        "optimizer_name": tune.choice(["adamw", "adam"]),
        "optimizer_params": {
            "weight_decay": tune.loguniform(1e-5, 1e-2),
        },

        # Scheduler params
        "scheduler_name": tune.choice(["reduce_on_plateau", "none"]),
        "scheduler_params": {
            "factor": tune.uniform(0.1, 0.5),
            "patience": tune.choice([3, 5, 10]),
        },
    }

    scheduler = ASHAScheduler(
        metric="val_acc_exact",
        mode="max",
        grace_period=10,
        reduction_factor=2,
    )

    # --- Add timestamp to run name ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"fnn_cnn_fnn_multilabel_tuning_{timestamp}"

    tuner = tune.Tuner(
        tune.with_resources(
            train_model_tune,
            {"cpu": 1, "gpu": 1},  # adjust as needed
        ),
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=20,
        ),
        run_config=air.RunConfig(
            name=run_name,
            storage_path="/home/iatell/projects/meta-learning/tune_logs",
        ),
    )

    results = tuner.fit()

    # Get the best trial
    best_result = results.get_best_result(metric="val_acc_exact", mode="max")
    print("\n🏆 Best Config:", best_result.config)
    print(f"Best Macro F1 Score: {best_result.metrics['val_acc_exact']:.4f}")

    # Retrain with best config
    if save_model:
        print("\n🔁 Retraining best model on full dataset for saving...")

        retrain_args = {
            "data_csv": data_csv,
            "labels_json": labels_json,
            "do_validation": False,
            "model_out_dir": "models/saved_models",
            "save_model": True,
            "dropout": best_result.config["dropout"],
            "lr": best_result.config["lr"],
            "batch_size": best_result.config["batch_size"],
            "max_epochs": best_result.config["max_epochs"],
            "optimizer_name": best_result.config["optimizer_name"],
            "scheduler_name": best_result.config["scheduler_name"],

            # Nested dicts
            "optimizer_params": {
                "weight_decay": best_result.config["optimizer_params"]["weight_decay"]
            },
            "scheduler_params": best_result.config["scheduler_params"],

            # Model-specific
            "fnn_hidden_dim": best_result.config["fnn_hidden_dim"],
            "output_channel": best_result.config["output_channel"],
        }

        train_model(**retrain_args)

if __name__ == "__main__":
    run_tuning(save_model=True)


2025-09-22 21:30:48,523	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/iatell/projects/meta-learning/tune_logs/fnn_cnn_multilabel_tuning_20250922_212647' in 0.0047s.
2025-09-22 21:30:48,529	INFO tune.py:1041 -- Total run time: 240.97 seconds (240.94 seconds for the tuning loop).



🏆 Best Config: {'fnn_hidden_dim': 128, 'output_channel': 32, 'dropout': 0.36937982977776407, 'lr': 0.0008391502158343265, 'batch_size': 64, 'max_epochs': 200, 'optimizer_name': 'adam', 'optimizer_params': {'weight_decay': 0.001680623888451716}, 'scheduler_name': 'none', 'scheduler_params': {'factor': 0.19065116293646933, 'patience': 3}}
Best Macro F1 Score: 0.6180

🔁 Retraining best model on full dataset for saving...


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params | Mode 
------------------------------------------------------------
0 | fnn_layer     | Sequential        | 1.0 K  | train
1 | cnn_extractor | Sequential        | 4.2 K  | train
2 | fc            | Linear            | 198    | train
3 | criterion     | BCEWithLogitsLoss | 0      | train
------------------------------------------------------------
5.4 K     Trainable params
0         Non-tra


=== DEBUG SAMPLE CHECK ===
Total sequences collected: 444

--- Sequence 0 ---
Original label(s): ['H']
Cleaned label(s): ['H']
Encoded: [0 1 0 0 0 0]
Feature shape: (1, 5)
First few timesteps:
 [[14210.    14339.5   12569.2   13474.99  17017.633]]



/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.



✅ Model saved to models/saved_models/lstm_model_class_20250922_213048.pt
✅ Meta saved to models/saved_models/lstm_meta_class_20250922_213048.pkl


## cnn fnn

In [ ]:
import os
import torch
from utils.resoure_usage import resource_usage
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray import air
from datetime import datetime

data_csv = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"
labels_json = "/home/iatell/projects/meta-learning/data/candle_labels.json"

def train_model_tune(config):
    """
    Single Ray Tune trial for CNN-FNN model.
    """
    # resource_usage()  # Uncomment to log hardware usage

    # Fixed args
    fixed_args = dict(
        data_csv=data_csv,
        labels_json=labels_json,
        do_validation=True,
        model_out_dir="models/tuned",
        return_val_accuracy=True,
        save_model=False,
    )

    # Start with fixed args
    train_args = fixed_args.copy()

    # 🌟 Core hyperparameters
    train_args["output_channel"] = config["output_channel"]
    train_args["dropout"] = config["dropout"]
    train_args["lr"] = config["lr"]
    train_args["batch_size"] = config["batch_size"]
    train_args["max_epochs"] = config["max_epochs"]

    # 🌟 Optimizer
    train_args["optimizer_name"] = config["optimizer_name"]
    train_args["optimizer_params"] = {
        "weight_decay": config["optimizer_params"]["weight_decay"]
    }

    # 🌟 Scheduler
    train_args["scheduler_name"] = config["scheduler_name"]
    if train_args["scheduler_name"] == "reduce_on_plateau":
        train_args["scheduler_params"] = {
            "factor": config["scheduler_params"]["factor"],
            "patience": config["scheduler_params"]["patience"],
        }
    else:
        train_args["scheduler_params"] = {}

    # Train
    metrics = train_model(**train_args)
    tune.report(metrics)

def run_tuning(save_model=False):
    """Hyperparameter tuning for CNN-FNN model with Ray Tune."""

    search_space = {
        # Model hyperparameters
        "output_channel": tune.choice([16, 32, 64, 128]),
        "dropout": tune.uniform(0.1, 0.5),

        # Optimizer / training
        "lr": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([32, 64, 128]),
        "max_epochs": tune.choice([100, 150, 200]),

        # Optimizer params
        "optimizer_name": tune.choice(["adamw", "adagrad"]),
        "optimizer_params": {
            "weight_decay": tune.loguniform(1e-5, 1e-2),
        },

        # Scheduler params
        "scheduler_name": tune.choice(["reduce_on_plateau", "none"]),
        "scheduler_params": {
            "factor": tune.uniform(0.1, 0.5),
            "patience": tune.choice([3, 5, 10]),
        },
    }

    scheduler = ASHAScheduler(
        metric="val_f1_macro",
        mode="max",
        grace_period=10,
        reduction_factor=2,
    )

    # Unique run name
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"cnn_fnn_multilabel_tuning_{timestamp}"

    tuner = tune.Tuner(
        tune.with_resources(train_model_tune, {"cpu": 1, "gpu": 1}),
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=20,
        ),
        run_config=air.RunConfig(
            name=run_name,
            storage_path="/home/iatell/projects/meta-learning/tune_logs",
        ),
    )

    results = tuner.fit()

    # Best result
    best_result = results.get_best_result(metric="val_f1_macro", mode="max")
    print("\n🏆 Best Config:", best_result.config)
    print(f"Best Macro F1 Score: {best_result.metrics['val_f1_macro']:.4f}")

    # Retrain with best config
    if save_model:
        print("\n🔁 Retraining best model on full dataset for saving...")

        retrain_args = {
            "data_csv": data_csv,
            "labels_json": labels_json,
            "do_validation": False,
            "model_out_dir": "models/saved_models",
            "save_model": True,

            # From best config
            "output_channel": best_result.config["output_channel"],
            "dropout": best_result.config["dropout"],
            "lr": best_result.config["lr"],
            "batch_size": best_result.config["batch_size"],
            "max_epochs": best_result.config["max_epochs"],
            "optimizer_name": best_result.config["optimizer_name"],
            "scheduler_name": best_result.config["scheduler_name"],

            # Nested dicts
            "optimizer_params": {
                "weight_decay": best_result.config["optimizer_params"]["weight_decay"]
            },
            "scheduler_params": best_result.config["scheduler_params"],
        }

        train_model(**retrain_args)

if __name__ == "__main__":
    run_tuning(save_model=True)


2025-09-22 21:10:13,922	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_tune_3003a_00000
Traceback (most recent call last):
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/_private/worker.py", line 2849, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

RuntimeError: No best trial found for the given metric: val_f1_macro. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

# tuning xg boost

In [ ]:
import os
import torch
from utils.flatten_config import flatten_config
from utils.resoure_usage import resource_usage
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray import air
from datetime import datetime

def train_model_tune(config):
    """
    Single Ray Tune trial for XGB Multilabel model.
    """
    resource_usage()  # Show current hardware usage

    # Always required data sources
    data_csv = "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv"
    labels_json = "/home/iatell/projects/meta-learning/data/candle_labels.json"

    # Fixed args
    fixed_args = dict(
        data_csv=data_csv,
        labels_json=labels_json,
        do_validation=True,
        model_out_dir="models/tuned",
        return_val_accuracy=True,
        save_model=False,
    )

    # Flatten (in case config has nested dicts)
    flat_config = flatten_config(config)

    # Merge everything
    train_args = {**fixed_args, **flat_config}

    # Train and collect metrics
    metrics = train_model_xgb_multilabel(**train_args)

    # Report to Ray Tune
    tune.report(metrics)


def run_tuning(save_model=False):
    """Hyperparameter tuning for XGB Multilabel with Ray Tune."""

    search_space = {
        # Sequence length
        # "seq_len": tune.choice([1, 3, 5, 10, 20]),

        # Core XGB hyperparameters
        "n_estimators": tune.choice([100, 200, 400, 800]),
        # "max_depth": tune.choice([3, 6, 9]),
        # "learning_rate": tune.loguniform(1e-3, 0.3),   # typical XGB LR range
        "subsample": tune.uniform(0.6, 1.0),
        "colsample_bytree": tune.uniform(0.6, 1.0),

        # Label handling
        # "label_weighting": tune.choice(["none", "scale_pos"]),
        # "include_no_label": tune.choice([False, True]),

        # Threshold tuning
        # "threshold_tuning": tune.choice([False, True]),
    }

    scheduler = ASHAScheduler(
        metric="exact_match",   # use micro accuracy from your return dict
        mode="max",
        grace_period=1,
        reduction_factor=2
    )
    # --- Add timestamp to run name ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"xgb_multilabel_tuning_{timestamp}"

    tuner = tune.Tuner(
        train_model_tune,
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=20,   # try more samples, adjust depending on compute
        ),
        run_config=air.RunConfig(
            name=run_name,
            storage_path="/home/iatell/projects/meta-learning/tune_logs",
        ),
    )

    results = tuner.fit()

    # Best trial
    best_result = results.get_best_result(metric="exact_match", mode="max")
    print("\n🏆 Best Config:", best_result.config)
    print(f"Best Micro Accuracy: {best_result.metrics['exact_match']:.4f}")
    # Optional: retrain best model on full data and save
    if save_model:
        print("\n🔁 Retraining best XGB model on full dataset for saving...")

        # Re-train using the best configuration found
        final_model_metrics = train_model_xgb_multilabel(
            data_csv="/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv",
            labels_json="/home/iatell/projects/meta-learning/data/candle_labels.json",
            do_validation=True,
            return_val_accuracy=True,
            model_out_dir="models/saved_models",
            save_model=True,  # <--- actually save model here
            seq_len=best_result.config.get("seq_len", 5),
            n_estimators=best_result.config.get("n_estimators", 200),
            max_depth=best_result.config.get("max_depth", 6),
            learning_rate=best_result.config.get("learning_rate", 0.05),
            subsample=best_result.config.get("subsample", 0.8),
            colsample_bytree=best_result.config.get("colsample_bytree", 0.8),
            label_weighting=best_result.config.get("label_weighting", "none"),
            include_no_label=best_result.config.get("include_no_label", False),
            threshold_tuning=best_result.config.get("threshold_tuning", False),
        )

        print("\n💾 Final saved model metrics:", final_model_metrics)
if __name__ == "__main__":
    run_tuning(save_model=False)


2025-09-22 21:49:21,396	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_tune_a6313_00019
Traceback (most recent call last):
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/ray/_private/worker.py", line 2849, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

# tensorboard model

In [ ]:
import os
import subprocess
import webbrowser

logdir = "lightning_logs"

# 1. Find all version folders
versions = [d for d in os.listdir(logdir) if d.startswith("version_") and d.split("_")[1].isdigit()]
if not versions:
    raise ValueError("No version folders found in lightning_logs")

# 2. Pick the latest numerically
latest_version = max(versions, key=lambda x: int(x.split("_")[1]))
latest_logdir = os.path.join(logdir, latest_version)
print(f"Launching TensorBoard for: {latest_logdir}")

# 3. Choose a port
port = 6006

# 4. Launch TensorBoard as a background process
subprocess.Popen(["tensorboard", f"--logdir={latest_logdir}", f"--port={port}"])

# 5. Open TensorBoard in default browser
webbrowser.open(f"http://localhost:{port}")



Launching TensorBoard for: lightning_logs/version_37


True

gio: http://localhost:6006: Operation not supported
2025-09-22 21:46:06.967861: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-22 21:46:06.979429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758564966.992070   52360 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758564966.995816   52360 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758564967.005953   52360 computation_placer.cc:177] computation placer already

# tensorboard tuning

In [ ]:
import os
import subprocess
import webbrowser

# Base Ray Tune log directory
base_logdir = "/home/iatell/projects/meta-learning/tune_logs"

# Prefix filter (set to "" to disable filtering)
# start_with = "xgb_multilabel_tuning_"   # e.g., only load runs that start with this
# start_with = "fullfnn_multilabel_tuning"
start_with =  "fnn_cnn_multilabel_tuning"
# 1. Find all experiment folders
experiments = [
    d for d in os.listdir(base_logdir)
    if os.path.isdir(os.path.join(base_logdir, d))
    and d.startswith(start_with)
]

if not experiments:
    raise ValueError(f"No experiment folders found in tune_logs starting with '{start_with}'")

# 2. Sort by modification time and get the latest experiment
experiments.sort(key=lambda x: os.path.getmtime(os.path.join(base_logdir, x)))
latest_experiment = experiments[-1]
latest_logdir = os.path.join(base_logdir, latest_experiment)
print(f"🚀 Launching TensorBoard for: {latest_logdir}")

# 3. Choose a port
port = 6006

# 4. Launch TensorBoard as a background process
subprocess.Popen([
    "tensorboard",
    f"--logdir={latest_logdir}",
    f"--port={port}"
])

# 5. Open TensorBoard in default browser
webbrowser.open(f"http://localhost:{port}")



🚀 Launching TensorBoard for: /home/iatell/projects/meta-learning/tune_logs/fnn_cnn_multilabel_tuning_20250922_212647


True

gio: http://localhost:6006: Operation not supported
2025-09-22 21:31:21.759277: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-22 21:31:21.768925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758564081.779705   52013 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758564081.782822   52013 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758564081.790998   52013 computation_placer.cc:177] computation placer already